In [1]:
import cv2
from ultralytics import YOLO

In [2]:
# yolov8s 모델 사용
model = YOLO('yolov8n.pt')

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="")  # Roboflow 계정의 API 키
project = rf.workspace().project("knife_object")  # 프로젝트 이름
dataset = project.version(1).download("yolov8")  # YOLOv8 형식으로 다운로드

In [ ]:
# 2. 학습 수행
model.train(
    data="/home/zeus/ws/AEGISVISION/src/weapon_detect_model/knife_object-1/data.yaml",  # Roboflow에서 다운로드한 data.yaml 경로
    epochs=120,                        # 학습 반복 횟수
    batch=16,                         # 배치 크기
    imgsz=640                         # 이미지 크기
)

# 3. 학습 완료된 모델 저장
model_path = "runs/detect/train/weights/best.pt"  # 학습된 모델 경로

In [ ]:
# 3. 학습 완료된 모델 저장
#model_path = "/home/zeus/ws/AEGISVISION/src/weapon_detect_model/runs/detect/train/weights/best.pt"  # 학습된 모델 경로

In [ ]:
model = YOLO("/home/zeus/ws/AEGISVISION/src/weapon_detect_model/runs/detect/train/weights/best.pt")

# 테스트 데이터로 평가
results = model.val(data="/home/zeus/ws/AEGISVISION/src/weapon_detect_model/knife_object-1/data.yaml")

# 평가 결과 출력
print(dir(results))  # Precision, Recall, mAP 등

In [ ]:
print(results)

In [7]:
import cv2
from ultralytics import YOLO

# 학습된 모델 로드
model = YOLO("/home/zeus/ws/AEGISVISION/src/weapon_detect_model/runs/detect/train/weights/best.pt")

In [ ]:
# 이미지 파일 경로
image_path = "/home/hun/ws/yolov8/image/knife15.png"

# 객체 탐지 수행
result = model.predict(source=image_path, save=True, conf=0.5)

plots = result[0].plot()
cv2.imshow("plot", plots)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 객체 탐지 수행
result = model.predict(source=image_path, save=True, conf=0.5)

plots = result[0].plot()
cv2.imshow("plot", plots)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("/home/zeus/ws/AEGISVISION/src/weapon_detect_model/runs/detect/train/weights/best.pt")

# 웹캠 또는 비디오 파일 읽기 (0은 기본 웹캠을 의미)
cap = cv2.VideoCapture(0)  # 비디오 파일을 사용하려면 "video.mp4"와 같은 파일 경로를 입력

# 비디오 캡처 확인
if not cap.isOpened():
    print("Error: Cannot access the camera or video file.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Cannot read frame.")
        break

    # YOLO 모델을 사용하여 객체 탐지 수행
    results = model(frame)

    # 탐지 결과를 프레임에 표시
    for result in results:
        # 탐지된 객체의 정보를 반복
        for box in result.boxes:
            # 바운딩 박스 좌표 및 클래스 정보 추출
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # 좌표
            conf = box.conf[0]  # 신뢰도
            cls = int(box.cls[0])  # 클래스 인덱스
            label = model.names[cls]  # 클래스 이름

            # 바운딩 박스 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            # 클래스 이름 및 신뢰도 표시
            cv2.putText(
                frame,
                f"{label} {conf:.2f}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2,
            )

    # 결과 프레임을 OpenCV 창에 표시
    cv2.imshow("YOLO Object Detection", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
